In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/My Drive')

from __future__ import division

### Import packages:

import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import scipy
from scipy import signal
from scipy import misc
import keras
from keras.datasets import mnist  #
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import torch
from torch.nn import functional as Tfunc
# import chainer
# import chainer.functions as ch
import progressbar
import time


#### Set parameters:

cp.random.seed(0)
thr = [100, 100]
lr = [0.5, 0.5]
lrt = 1
b=[10,50]
a=[0,10]
Nepoch = 100

NumOfCatgry = 10
Nlayers = 2
NhidenNeurons = 350
Num_neurons = 784
dropout = [50, 0]
lamda = [0, 0]  #[0.000001,0.000001]
Nnrn = np.array([NhidenNeurons, NumOfCatgry])
Tmax = 101
Tend = 100
augment_size = 0
add = 15
neg = 15

Nepoch = 200
Tou = 100
tou1 = 80
# tou2 = Tou - tou1

add = 15
neg = 15
gamma = 3
best = 0  #0.9742

### Preparing filters:

th=[]
to=[]

layer = 0
W0 = (b[layer] - a[layer]) * cp.random.random_sample((NhidenNeurons,Num_neurons,1)) + a[layer]
layer = 1
W1 = (b[layer] - a[layer]) * cp.random.random_sample((NumOfCatgry,NhidenNeurons,1)) + a[layer]

### Data augmentation:

x_train, y_train, x_test, y_test = None, None, None, None
train_size, test_size = 0, 0
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# reshape
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
# convert from int to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# rescale values
# x_train /= 255.0
# x_test /= 255.0
# Save dataset sizes
train_size = x_train.shape[0]
test_size = x_test.shape[0]
        # Create one hot array
#         self.y_train = to_categorical(self.y_train, 10)
#         self.y_test = to_categorical(self.y_test, 10)
 
image_generator = ImageDataGenerator(
            rotation_range=5,  #10,
            zoom_range = 0.01,  #0.05, 
            width_shift_range=0.01,  #0.05,
            height_shift_range=0.01,  #0.05,
            horizontal_flip=False,
            vertical_flip=False, 
            data_format="channels_last",
            zca_whitening=False)
# fit data for zca whitening
image_generator.fit(x_train, augment=True)
# get transformed images
randidx = np.random.randint(train_size, size=augment_size)
x_augmented = x_train[randidx].copy()
y_augmented = y_train[randidx].copy()
x_augmented = image_generator.flow(x_augmented, np.zeros(augment_size),
                                    batch_size=augment_size, shuffle=False).next()[0]
# append augmented data to trainset
x_train = np.concatenate((x_train, x_augmented))
y_train = np.concatenate((y_train, y_augmented))
train_size = x_train.shape[0]
test_size = x_test.shape[0]

### Load data:

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Images = x_train
Labels = y_train
Images = np.array(Images)
images = []
labels = []
for i in range(len(Labels)):
    if Labels[i] < NumOfCatgry :
        images.append(np.floor(Tend-(np.dot(Images[i],Tend)/255).reshape(1,1,28,28))) #.astype(int))
        # images.append(np.floor(Tend-(np.dot(Images[i],Tend)/255).transpose(2,0,1).reshape(1,N_ch,Size_i,Size_i)))
        labels.append(Labels[i])          


Images = x_test
Labels = y_test
Images = np.array(Images)
images_test = []
labels_test = []
for i in range(len(Labels)):
    if Labels[i] < NumOfCatgry:
        images_test.append(np.floor(Tend-(np.dot(Images[i],Tend)/255).reshape(1,1,28,28))) #.astype(int))
        # images_test.append(np.floor(Tend-(np.dot(Images[i],Tend)/255).transpose(2,0,1).reshape(1,N_ch,Size_i,Size_i)))
        labels_test.append(Labels[i])



images = cp.asarray(images)
images_test = cp.asarray(images_test)
labels = cp.asarray(labels)
labels_test = cp.asarray(labels_test)

### Functions:

############## Define forward Fully Connected function: ##############
def Forward_FC(FT, weight, Num_in, Num_out, init, t):
    if (init==1):
        t = cp.mgrid[0:Tmax,0:Num_in,0:1][0][:,cp.newaxis]
    else:
        t = t     
    Voltage = cp.zeros(([Num_out,Tmax+1])) 
    hasfired1 = (t<=(FT+tou1)) * (FT<=t)
    # hasfired2 = (t>(FT+tou1)) * (t<=(FT+Tou))
    Voltage[:,0:Tmax] = ((cp.sum(weight*((t-FT)/tou1)*hasfired1,2)).T)[0] #+
                          # cp.sum(weight*((Tou/tou2)-((t-FT)/tou2))*hasfired2,2)).T)[0]
    Voltage[:,Tmax] = thr[layer] + 1
    ST = (cp.argmax(Voltage>=thr[layer],axis=1).astype(float)).reshape(Num_out,1) 

    return (ST, Voltage, t)

############## Define target function: ##############
def target_function(inp, Vinp, correct):

    target = cp.zeros([NumOfCatgry,1])
    winner = cp.argmin(inp)
    Vwin = cp.argmax(Vinp[:,inp[winner].astype(int)])        
    EQ = inp==inp[winner]  
    temp = inp[winner]

    minFiring = min(inp)
    maxFiring =max(inp)               
    if temp == Tmax:
        target[:]=Tmax
        target[labels[IMGs[iteration]]]=temp-add
        target=target.astype(int)
    else:
        target[:]=inp
        minFiring=min(inp)
        maxFiring=max(inp)
        if winner == labels[IMGs[iteration]]:
            if (cp.sum(EQ)>1):
                target[EQ] = min(minFiring+add,Tmax)    #target[:] = min(maxFiring+pos,Tmax)  #    #
                target[labels[IMGs[iteration]]] = max(minFiring-neg,0)
                pass
            else:
                pass
        else:
            if minFiring==maxFiring:
                target[EQ] = min(minFiring+add,Tmax)
                target[labels[IMGs[iteration]]] = max(minFiring-neg,0)
            else:
                if (inp[labels[IMGs[iteration]]]==inp[winner]):
                    #target[:] = min(minFiring+add,Tmax)  ### add it new
                    target[EQ] = min(minFiring+add,Tmax)   # min(minFiring+add,maxFiring)   #              
                    target[labels[IMGs[iteration]]]=max(minFiring-neg,0)
                else:
                    #target[:] = min(minFiring+add,Tmax)  ### add it new
                    target[Vwin] = min(minFiring+add,maxFiring)             
                    target[labels[IMGs[iteration]]] = max(minFiring-neg,0)

    if minFiring==Tmax:
        Vwin1=cp.argmax(Vinp[:,Tmax-3])
        if Vwin1==labels[IMGs[iteration]]:
            correct+=1
    else:            
        if EQ[labels[IMGs[iteration]]]:
            if (winner==labels[IMGs[iteration]]):
                correct+=1
            elif (Vwin==labels[IMGs[iteration]]):
                correct+=1  

    return(target,correct)

############## Dropout function: ##############
def DP(dp, inp, Num_in):
    if (dp>0):
        inp[cp.asarray(cp.random.permutation(Num_in)[:dp])] = Tmax+1  
    return(inp)

############## Training of the Fully Connected: ##############
def backward_FC(delta, inp, out, weight, lr):        

    hasFired = (inp<=out) 
    hasFired1 = ((out<=inp+tou1))
    # hasFired2= (((out>inp+tou1) * (out<=inp+Tou)))
    ft = (lr*delta)[:,cp.newaxis]
    ftP = (hasFired1*hasFired*(((out-inp)/tou1)))
    # ftN = (hasFired2*hasFired*((Tou/tou2)-(((out-inp)/tou2))))
    weight = weight - ft*ftP #- ft*ftN
    return(weight,hasFired,hasFired1)  #,hasFired2)

############## Find the best presynaptic spike times: ##############    
def Pre_ST(Beta, delta, weight, hasFired, hasFired1):  #, hasFired2):    

    ft = (delta*Beta)[:,cp.newaxis] 
    ftP = (hasFired1*hasFired/tou1)
    # ftN = (hasFired2*hasFired/tou2)      
    sumh = + (weight*ft)*ftP #- (weight*ft)*ftN
    return(sumh)


### The main code:

import time
init=1

for epoch in range(Nepoch):
    correct=0.0
    IMGs = cp.array(range(60000))   #cp.random.randint(0, 60000+augment_size, 60000, dtype=int)
    for iteration in progressbar.progressbar(range(len(IMGs))):
######################################################## 
##################### Forward processing: ##############  
########################################################
        ############# Fully connected layer: ##############
        layer = 0               
        FiringTime0 = cp.reshape(images[IMGs[iteration]],[Num_neurons,1])        
        (FiringTime1,VoltageH,th) = Forward_FC(FiringTime0, W0, Num_neurons, NhidenNeurons, init, th)
        FiringTime1[FiringTime1==Tmax] = Tmax+1
        layer = 1
        (SpikeTime,VoltageO,to) = Forward_FC(FiringTime1, W1, NhidenNeurons, NumOfCatgry, init, to)

        ############# Find the error and target time: ##############

        (target, correct) = target_function(SpikeTime, VoltageO, correct)

####################################################################
##################### Backward processing (Training): ##############
####################################################################
        ############# Fully connected layer: ##############
        #########
        layer= 1          
        delta1=((target-SpikeTime)/Tmax)
        norm = cp.linalg.norm(delta1)
        if (norm!=0):
            delta1 = delta1/norm       

        FiringTime1 = DP(dropout[layer-1], FiringTime1, NhidenNeurons)
        (W1,hasFired,hasFired1) = backward_FC(delta1,FiringTime1,SpikeTime[:,cp.newaxis],W1,lr[layer])   #,hasFired2
        W1 -= lr[layer]*lamda[layer]*W1    
        sumh1 = Pre_ST(1, delta1, W1, hasFired, hasFired1)#, hasFired2)
        #########
        layer= 0  
        delta_t0 = cp.sum(sumh1,0)
        norm = cp.linalg.norm(delta_t0)
        if (norm!=0):
            delta_t0 = delta_t0/norm  
        pre_Target = cp.zeros((NhidenNeurons,1))
        pre_Target = FiringTime1 + delta_t0
        pre_Target[pre_Target<=0] = 0
        pre_Target[pre_Target>=Tmax+1] = Tmax+1
        delta0 = ((pre_Target-FiringTime1))

        norm = cp.linalg.norm(delta0)
        if (norm!=0):
            delta0 = delta0/norm       
        
        # FiringTime1[FiringTime1==Tmax+1] = Tmax
        FiringTime0 = DP(dropout[layer-1], FiringTime0, Num_neurons) 
        (W0,hasFired,hasFired1) = backward_FC(delta0,FiringTime0,FiringTime1[:,cp.newaxis],W0,lr[layer])  #,hasFired2
        W0 -= lr[layer]*lamda[layer]*W0 
        
        init = 0
    print()
    print('perf = ', correct/len(IMGs))


        ######## TEST: ########

    correct=0
    for iteration in progressbar.progressbar(range(len(images_test))):   #for iteration in range(len(images_test)): 
        ############# Fully connected layer: ##############
        layer = 0               
        FiringTime0 = cp.reshape(images_test[iteration],[Num_neurons,1])        
        (FiringTime1,VoltageH,th) = Forward_FC(FiringTime0, W0, Num_neurons, NhidenNeurons, init, th)
        FiringTime1[FiringTime1==Tmax] = Tmax+1
        layer = 1
        (SpikeTime,VoltageO,to) = Forward_FC(FiringTime1, W1, NhidenNeurons, NumOfCatgry, init, to)


        ############## Find the error and target time: ##############
        winner = cp.argmin(SpikeTime)
        Vwin = cp.argmax(VoltageO[:,SpikeTime[winner].astype(int)])        
        EQ = SpikeTime==SpikeTime[winner]  
        temp = SpikeTime[winner]
        minFiring = min(SpikeTime)
        maxFiring = max(SpikeTime)  

        if minFiring==Tmax:
            V=cp.argmax(VoltageO[:,Tmax-3])
            if V==labels_test[iteration]:
              correct+=1
        else:
            if SpikeTime[labels_test[iteration]]==minFiring:
                correct+=1       
 

    print()
    # print('### epoch = ', epoch)
    print('val = ', correct/len(images_test))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
